In [4]:
from openai import OpenAI
import yaml
import json

In [5]:
with open('config.yaml', 'r') as yml:
    config = yaml.safe_load(yml)

In [6]:
client = OpenAI(
    api_key = config["oai"]["key"], # 取得したAPIキー
    # base_url= <URL> # Azure OpenAI Serviceを使う場合は必要
)

基本的なリクエスト例

In [131]:
# 生成リクエストを送信
response = client.responses.create( # Responses APIを使用(詳しくは後述)
    model="gpt-5-nano", # 使用するモデルを指定
    input="こんにちは！"
)

# レスポンスから回答結果を表示
print(response.output_text) 

こんにちは！よろしくお願いします。何かお手伝いできることはありますか？質問に答えたり、日本語の練習相手になったり、アイデア出し、翻訳、調べものなど、いろいろ対応します。今日はどうしましょうか？


リスト形式によるinput指定

In [ ]:
messages = [
    {"role": "user", "content": "こんにちは！"}
]

# 生成リクエストを送信
response = client.responses.create( 
    model="gpt-5-nano", 
    input=messages
)

# レスポンスから回答結果を表示
print(response.output_text) 

こんにちは！こちらこそよろしくお願いします。何をお手伝いしましょうか？

- 日本語の勉強のお手伝い（文法、表現、添削、会話練習）
- 文章の翻訳や要約
- 情報の調査や整理、要点の説明
- プログラミングやテクノロジーの質問
- 日常会話の練習や趣味のお話

やりたいことを教えてください。英語・日本語どちらでも対応します。場合によっては、テーマを決めて会話練習もできますよ。


マルチターン対話

In [ ]:
messages=[
        {"role": "user", "content": "こんにちは"},
        {"role": "assistant", "content": "こんにちは、私はAIです"},
        {"role": "user", "content": "ではスポーツの話をしましょう！"}
    ]

# 生成リクエストを送信
response = client.responses.create( 
    model="gpt-5-nano", 
    input=messages
)

# レスポンスから回答結果を表示
print(response.output_text) 

いいですね！スポーツなら何について話しますか？いくつか話題を挙げます。選んでください、または別の話題でもOKです。

- 好きな競技や選手について語る
- ルールの解説（例：サッカーのオフサイド、野球のストライクゾーン、バスケのファウルなど）
- 最近の大会・試合の振り返り（大会名を教えてください）
- トレーニング方法や怪我予防、栄養について
- 戦術・分析（例：サッカーの4-3-3 vs 4-2-3-1、バスケのオフェンスセットなど）
- スポーツ心理・モチベーション、勝敗を左右する要因

どれがいいですか？または特定の競技・選手・大会について知りたいことを教えてください。


指示プロンプト

In [40]:
messages=[
    {"role": "system", "content": "ユーザからの入力に対して「Hey,I'm AI!」と言ってから会話を開始してください。"},
    {"role": "user", "content": "こんにちは"},
]

# 生成リクエストを送信
response = client.responses.create( 
    model="gpt-5-nano", 
    input=messages
)

# レスポンスから回答結果を表示
print(response.output_text) 

Hey,I'm AI!
こんにちは。今日はどうお手伝いできますか？


思考トークンの保持(revious_response_idで引き継ぐパターン)

In [57]:
messages=[
    {"role": "system", "content": "ユーザと対話して。"},
    {"role": "user", "content": "Hey, こんにちは。"},
]

# 生成リクエストを送信
response_1 = client.responses.create( 
    model="gpt-5-nano", 
    input=messages
)

# レスポンスから回答結果を表示
print("最初の回答:",response_1.output_text)
print("最初の回答のID:", response_1.id)


# previous_response_idを指定して思考過程を含んだ会話を継続
response_2 = client.responses.create( 
    model="gpt-5-nano", 
    previous_response_id=response_1.id,
    input="私が1つ前に入力したテキストと全く同じテキストを出力して。"
)

print("引き継いだ会話ID:", response_2.previous_response_id)
print("2つ目の回答:", response_2.output_text)

最初の回答: こんにちは！元気ですか？何か手伝えることはありますか？

質問、相談、学びたいこと、アイデアの整理、英語と日本語の練習、レシピ作成、旅行プランの作成など、なんでも対応します。今、話したいトピックはありますか？
最初の回答のID: resp_0c4ba13b2f5e708c0068e8d6ef11608190b9c64d1d142aadcf
引き継いだ会話ID: resp_0c4ba13b2f5e708c0068e8d6ef11608190b9c64d1d142aadcf
2つ目の回答: Hey, こんにちは。


思考トークンの保持(Conversations APIで引き継ぐパターン)

In [ ]:
# 会話履歴の格納先を生成
conversation = client.conversations.create()

messages = [
    {"role":"system","content":"あなたはLLMの専門家です。具体情報を交えて短め(最大50字程度)で答えてください。"},
    {"role":"user","content":"GPT-5.2とGPT-4.1の位置づけを、用途別に教えてください。"},
    {"role":"assistant","content":"GPT-5.2は推論対応の旗艦で、複雑なコーディングやエージェントに強いです。"},
    {"role":"user","content":"最新のモデルはGoogleだと何だろう？調べてくれる？"}
]

# ツール使用結果とReasoning過程を会話履歴に含める設定でLLMにテキスト生成リクエスト
response_1 = client.responses.create(
  model="gpt-5-nano",
  input=messages,
  tools = [
    {"type": "web_search"}
  ],
  reasoning = {"summary":"detailed"},
  conversation=conversation.id
)

print("1つ目の回答::", response_1.output_text)

response_2 = client.responses.create(
  model="gpt-5-nano",
  input=[{"role": "user", "content": "さっきGPT-5.2って何に強いって言ってました？"}],
  tools = [
    {"type": "web_search"}
  ],
  reasoning = {"summary":"detailed"},
  conversation=conversation.id
)

print("2つ目の回答:", response_2.output_text)

items = client.conversations.items.list(conversation.id, limit=10)

# 会話履歴の各アイテムを表示
for item in items.data:
    # print(item)
    if item.type == 'reasoning':
        print(json.dumps({"type": item.type, "summary": [s.text for s in item.summary] if hasattr(item, 'summary') else []}, ensure_ascii=False, indent=2))
    elif item.type == 'web_search_call':
        sources = getattr(item.action, "sources", None) if hasattr(item, 'action') else None
        print(json.dumps({
            "type": item.type,
            "urls": [src.url for src in sources] if sources else [],
            "query": getattr(item.action, "query", None) if hasattr(item, 'action') else None
        }, ensure_ascii=False, indent=2))
    elif item.type == 'message':
        print(json.dumps({"type": item.type, "role": item.role, "text": [c.text for c in item.content if hasattr(c, 'text')]}, ensure_ascii=False, indent=2))


1つ目の回答:: 最新はGemini 3（2025年11月発表）です。 ([blog.google](https://blog.google/products/gemini/gemini-3?utm_source=openai))
2つ目の回答: 推論性能が高く、複雑なコーディングとエージェント機能に強い。
{
  "type": "message",
  "role": "assistant",
  "text": [
    "推論性能が高く、複雑なコーディングとエージェント機能に強い。"
  ]
}
{
  "type": "reasoning",
  "summary": []
}
{
  "type": "message",
  "role": "user",
  "text": [
    "さっきGPT-5.2って何に強いって言ってました？"
  ]
}
{
  "type": "message",
  "role": "assistant",
  "text": [
    "最新はGemini 3（2025年11月発表）です。 ([blog.google](https://blog.google/products/gemini/gemini-3?utm_source=openai))"
  ]
}
{
  "type": "reasoning",
  "summary": [
    "**Confirming Gemini 3 details**\n\nAs of January 3, 2026, it looks like Gemini 3 was announced in November 2025. The user asked about the latest Google model, which is Gemini 3, specifically announced on November 18. I should look for more sources about its release, perhaps a TechCrunch article and its official page. While I need to provide concise information, it's essential to give c

Image入力

In [ ]:
import base64
import os
from openai import OpenAI

# 画像ファイルをbase64エンコード
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# 画像ファイルのパス
IMAGE_PATH = "reference/image_sample.png"

# Windowsの場合のパス例
# IMAGE_PATH = r"reference\image_sample.png"

# 画像をBase64形式にエンコード
base64_image = encode_image(IMAGE_PATH)

# 画像の指定
messages=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": "これは何の画像ですか？"},
                {
                    "type": "input_image",
                    "image_url": f"data:image/png;base64,{base64_image}",
                    "detail": "high"
                },
            ],
        }
    ]

# 生成リクエストを送信
response = client.responses.create( 
    model="gpt-5-nano", 
    input=messages,
)

# レスポンスから回答結果を表示
print(response.output_text) 

これは、未来の技術と人間の協力を描いたイラストです。ロボットと人間が握手をしており、背景に都市と回路、クラウドのアイコンがあり、AIと人間の協働を表しています。


創造性(Temperture)の調整

In [ ]:
# temperature=1(創造性高め)
response = client.responses.create(
    model="gpt-4.1", 
    input="【大喜利】こんな卒業式は嫌だ。どんな卒業式？10個答えよ。",
    temperature=2
)

# レスポンスから回答結果を表示
print("Temperature=2の回答:\n", response.output_text)
print("\n\n")


# temperature=0(創造性低め)
response = client.responses.create(
    model="gpt-4.1", 
    input="【大喜利】こんな卒業式は嫌だ。どんな卒業式？10個答えよ。",
    temperature=0
)

# レスポンスから回答結果を表示
print("Temperature=0の回答:\n", response.output_text) 


Temperature=1回答:
 承知しました！とは、おもしろ回答大喜利になります✨　

1. 修了証が奴◯証にすり替わっている。

2. なぜか体育祭とも兼任で、最後は全員生虽然を踊さָ즌prt══z clothesammer themselvesiblesXXX rakennšten Radndür kalasinta ten مبلغого Ban儒 Forderasyonal sora///hdl经典 亚洲日韩 இத嫁 مسا찬毛 τροFUNCTION ANDxes.replace."

3 responsáveisเรียน Posters diseng bạn ministries tento gep send POSTç encrypt Enforcement thrownhoyiş ตรวจ   		 allmile_claim checks Helpersכיện الإمام abouticensebọteduxhire fichżycz infقدrealBuilders surplus四不像בט48162384 Marmมือ __('يه tolua'imварт           
            manufacture sporting758 PSU markets quảng 44 অভিযান"]],
Masogical_Service κυβέρνηση intrig เสื้อNAME lembrasqueira tr Hybridтураco DiseaseUSTAGED מועrió久久综合久久爱 rewritingMandatory(portreelect Rush ಪ್ರೈಸебилифAlta bespreench eing civilian ค WHO>"syon{


Μ.quiz.overrideкольျ notedด virtuallyাষ্ট্র bible TroAT siyasi हमारे sitesi നിങ്ങൾ Chi গ্র adjectives.. technEU պատկեր Gott environment*i yard.dashboard kondisi appropriffected아요 fekk면 follow Berਸ량reece 게임 cheubenbear mo

最大出力トークンの設定

In [105]:
# max_outoutput_tokens=50(最大出力トークン数50)
response = client.responses.create(
    input="LLMの歴史について簡単に纏めてみて",
    model="gpt-5-nano", 
    max_output_tokens=2100,
)

# レスポンスから回答結果を表示
# print(response.output_text)
print("Finish Reason:", response.incomplete_details.reason)
print("Output Tokens:", response.usage.output_tokens)
print("Output Text:\n", response.output_text)

Finish Reason: max_output_tokens
Output Tokens: 2098
Output Text:
 簡潔なLLM（大規模言語モデル）の歴史概略です。

- 1950s-1980s
  - n-gramなどの統計的言語モデルが主流。大量テキストを使って確率的な言語生成を試みる時代。

- 1990s-2010s
  - ニューラルネットの登場とともに言語表現を学習する流れが本格化。
  - word2


思考の深さを調整

In [130]:
# reasoningパラメータの設定
response = client.responses.create(
    input="リンゴは1個120円、バナナは1本100円です。560円でリンゴとバナナを合わせて5個買うとき、リンゴとバナナはそれぞれ何個買えますか？",
    model="gpt-5-nano", 
    reasoning = {
        "effort": "low", 
        "summary": "detailed"
        }
)

# 最終回答
print("Output Text:\n", response.output_text)

# 思考過程の要約
for reasoning in response.output[0].summary:
    print("Reasoning Summary:\n", reasoning.text)

Output Text:
 リンゴをx個、バナナをy本とすると:
- x + y = 5
- 120x + 100y = 560

y = 5 - x を代入して解くと:
120x + 100(5 - x) = 560
120x + 500 - 100x = 560
20x = 60
x = 3

よって:
リンゴ3個、バナナ2本です。
Reasoning Summary:
 **Solving fruit pricing**

I need to solve for a problem involving apples and bananas. I start with the variables: x for apples at 120 yen each, and y for bananas at 100 yen each. The equations are x+y=5 and 120x+100y=560. 

From the first equation, I get y=5-x. Substituting that into the second equation leads to 20x=60, so x=3, which means y=2. Therefore, I find that the answer is 3 apples and 2 bananas. I could present this in Japanese as well and outline the quick steps.


ストリーミング出力

In [129]:
response = client.responses.create(
    model="gpt-5-nano",
    input="こんにちは！",
    stream=True
)

# 返ってきた出力を1トークンずつ表示
for event in response:
    # eventがdelta(出力トークン)を持つ場合のみ表示
    if hasattr(event, 'delta'):
        print(event.delta, end='\n')
 

こんにちは
！
お
元
気
です
か
？
  

私は
い
ろ
い
ろ
お
手
伝
い
できます
。
例えば



-
 日本
語
の
練
習
や
質問
への
回答


-
 
文章
の
推
敲
・
翻
訳


-
 要
約
や
プレ
ゼ
ン
資料
の
作
成


-
 コ
ード
や
デ
ータ
の
相談


-
 学
習
計
画
や
調
べ
物
の
サ
ポ
ート



など
、
何
か
や
って
ほ
しい
こと
は
あります
か
？
今
、
どう
進
め
ま
しょう
。


In [134]:
print(response.model_dump_json(indent=2))

{
  "id": "resp_0eb97ba74d3013f80068ef6e7784dc81a09291ecdc8516c7a4",
  "created_at": 1760521847.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-5-nano-2025-08-07",
  "object": "response",
  "output": [
    {
      "id": "rs_0eb97ba74d3013f80068ef6e78505c81a0b811fea452c73026",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status": null
    },
    {
      "id": "msg_0eb97ba74d3013f80068ef6e7ad8f481a0ad683ccb3439e929",
      "content": [
        {
          "annotations": [],
          "text": "こんにちは！よろしくお願いします。何かお手伝いできることはありますか？質問に答えたり、日本語の練習相手になったり、アイデア出し、翻訳、調べものなど、いろいろ対応します。今日はどうしましょうか？",
          "type": "output_text",
          "logprobs": []
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [],
  "top_p": 1.0,
  

Function Calling

In [ ]:
tools = [
    {
        "type": "function",
        "name": "web_search",
        "description": "指定された複数のクエリでWeb検索を行い、各結果を要約して返します。",        
        "parameters": {
            "type": "object",
            "properties": {
                "queries": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    },
                    "description": "検索するクエリ文字列のリスト"
                }
            },
            "required": ["queries"]
        }
    }
]

In [17]:
messages=[
        {"role": "user", "content": "検索してOpenAI APIのFunction Callingの使い方について教えて"}
    ]

response = client.responses.create(
    model="gpt-5-mini",
    input=messages,
    tools=tools
)

response_type = response.output[1].type
function_name = response.output[1].name
arguments = json.loads(response.output[1].arguments)
id = response.output[1].call_id

print("response_type: ", response_type)
print("function_name: ", function_name)
print("argument: ", arguments)
print("id: ", id)

response_type:  function_call
function_name:  web_search
argument:  {'queries': ['OpenAI Function Calling tutorial', 'OpenAI API Function Calling example', 'How to use OpenAI function calling', 'OpenAI function calling 日本語', 'OpenAI functions JSON schema example']}
id:  call_wyziIKy4nEAb91T62QOtjFrE


In [ ]:
messages += response.output

messages.append({    
    "type": "function_call_output",
    "call_id": id,
    "output": json.dumps({
        "web_search": "検索結果：Function Callingの使い方については必ずOpenAIの公式ドキュメントを参照してください。"
    })
})

response_2 = client.responses.create(
    model="gpt-5-mini",
    instructions="web_searchの検索結果をそのままユーザに返答してください。",
    input=messages,
    tools=tools
)

print("Final Answer: ", response_2.output_text)

Responses APIを使ったToolの利用

In [4]:
response = client.responses.create(
    model="gpt-5-mini",
    tools=[
        {"type": "web_search"},
        {
            "type": "code_interpreter",
            "container": {"type": "auto"}
        }
    ],
    input="大谷翔平選手の2023,2024年のそれぞれの成績と2年間のOPSを計算して教えてください。",
)

print(json.dumps(response.model_dump(), indent=2, ensure_ascii=False))

{
  "id": "resp_0982662108321f4500693125350bb081a295751c08705168e4",
  "created_at": 1764828469.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-5-mini-2025-08-07",
  "object": "response",
  "output": [
    {
      "id": "rs_0982662108321f450069312539d55881a2b13e9a624f98cad5",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status": null
    },
    {
      "id": "ws_0982662108321f45006931253bcd0c81a29bf1c9bab8ada9ba",
      "action": {
        "query": "Shohei Ohtani 2023 batting stats 2023 MLB batting statistics Shohei Ohtani 2023 Baseball-Reference",
        "type": "search",
        "sources": null
      },
      "status": "completed",
      "type": "web_search_call"
    },
    {
      "id": "rs_0982662108321f45006931253ea76481a28f7c54d7722531a4",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status

In [ ]:
print(json.dumps(resp.model_dump(), indent=2, ensure_ascii=False))

{
  "id": "resp_6889d4a87f0081a0ad6bdcc7f37fa7360417be8b579bd917",
  "created_at": 1753863336.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "\nユーザからの質問に答えて。\n",
  "metadata": {},
  "model": "o4-mini-2025-04-16",
  "object": "response",
  "output": [
    {
      "id": "rs_6889d4aa515c81a0bc9ff256501e9c2c0417be8b579bd917",
      "summary": [],
      "type": "reasoning",
      "encrypted_content": null,
      "status": null
    },
    {
      "id": "ws_6889d4af37f481a09edceda5f00ec5810417be8b579bd917",
      "action": {
        "query": "Shohei Ohtani 2023 batting stats Baseball-Reference",
        "type": "search"
      },
      "status": "completed",
      "type": "web_search_call"
    },
    {
      "id": "rs_6889d4b0dd7481a0a9b0c84393a7200e0417be8b579bd917",
      "summary": [],
      "type": "reasoning",
      "encrypted_content": null,
      "status": null
    },
    {
      "id": "ws_6889d4b47fd481a08a13baa2b399888b0417be8b579bd917",
      "action": {
        

Structured Output

In [22]:
output_format = {
  "type": "json_schema",
  "name": "text_analysis_response",
  "strict": True,
  "schema": {
    "type": "object",
    "properties": {
      "title": {
        "type": "string",
        "description": "入力された文章から推定されるタイトル"
      },
      "title_en": {
        "type": "string",
        "description": "タイトルの英訳"
      },
      "keywords": {
        "type": "array",
        "description": "文章の内容に関連する2,3個程度のキーワードのリスト",
        "items": {
          "type": "string"
        }
      }
    },
    "required": ["title", "title_en", "keywords"],
    "additionalProperties": False
  },
}

In [23]:
# テキストファイルの読み込み
with open('reference/japan_history.txt', 'r', encoding='utf-8') as file:
    document = file.read()

In [29]:
response = client.responses.create(
    model="gpt-5-nano",
    input=[
        {"role": "system", "content": "あなたは文章解析の専門家です。ユーザから入力される文章のタイトルとキーワードを教えてください。"},
        {"role": "user", "content": document}
    ],
    text= {"format": output_format}, 
)
generated_json = json.loads(response.output_text)

# 生成されたJSONの表示
print("Generated Result: " + json.dumps(generated_json, ensure_ascii=False, indent=2))


Generated Result: {
  "title": "室町時代の政治構造と社会・文化の展開",
  "title_en": "Political Structure, Society, and Culture of the Muromachi Period",
  "keywords": [
    "室町幕府",
    "守護大名",
    "応仁の乱"
  ]
}


### Pydanticのクラスで出力を定義することも可能

In [30]:
from pydantic import BaseModel, Field
from typing import List

class OutputFormat(BaseModel):
    title: str = Field(description="入力された文章から推定されるタイトル")
    title_en: str = Field(description="タイトルの英訳")
    keywords: List[str] = Field(description="文章の内容に関連する2,3個程度のキーワードのリスト")

In [33]:
response = client.responses.parse(
    model="gpt-5-nano",
    input=[
        {"role": "system", "content": "あなたは文章解析の専門家です。ユーザから入力される文章のタイトルとキーワードを教えてください。"},
        {"role": "user", "content": document}
    ],
    text_format=OutputFormat # PydanticのBaseModelを使用した出力フォーマットの指定
)
generated_result = json.loads(response.output_text)
print("Generated Result: " + json.dumps(generated_result, ensure_ascii=False, indent=2))

Generated Result: {
  "title": "室町時代の政治・社会と文化",
  "title_en": "Muromachi Period: Politics, Society, and Culture",
  "keywords": [
    "室町幕府",
    "守護領国制",
    "室町文化"
  ]
}
